In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
sns.set_style('darkgrid')

In [4]:
import nltk
import re

In [5]:
import string

In [6]:
from nltk.corpus import stopwords

In [7]:
df = pd.read_csv('modelSelection/mbti_1.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


## Add columns for the type indicatiors

In [8]:
def get_types(row):
    t=row['type']

    I = 0; N = 0
    T = 0; J = 0
    
    if t[0] == 'I': I = 1
    elif t[0] == 'E': I = 0
    else: print('I-E incorrect')
        
    if t[1] == 'N': N = 1
    elif t[1] == 'S': N = 0
    else: print('N-S incorrect')
        
    if t[2] == 'T': T = 1
    elif t[2] == 'F': T = 0
    else: print('T-F incorrect')
        
    if t[3] == 'J': J = 1
    elif t[3] == 'P': J = 0
    else: print('J-P incorrect')
    return pd.Series( {'IE':I, 'NS':N , 'TF': T, 'JP': J }) 

### Label indicatiors
* Introvert 1 | Extrovert 0
* Intuition 1 | Sensing 0
* Thinking 1 | Feeling 0
* Judging 1 | Perciving 0

In [9]:
df = df.join(df.apply (lambda row: get_types (row),axis=1))
df.head(5)

,type,posts,IE,NS,TF,JP
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,1,1,0,1
1,ENTP,'I'm finding the lack of me in these posts ver...,0,1,1,0
2,INTP,'Good one _____ https://www.youtube.com/wat...,1,1,1,0
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",1,1,1,1
4,ENTJ,'You're fired.|||That's another silly misconce...,0,1,1,1


In [10]:
# processing the data
def preprocess_string(x_str, return_joined=True):
    '''Returns a cleaned string specifically from the MBTI dataset.
    If return_joined is True, the tokens are joined into a single string so
    that it can be passed into SciKit learn's frequency counter. Otherwise
    the tokens are returned as a list.'''

    # lower
    x_str = x_str.lower()

    # remove |||
    x_str = re.sub("[]|||[]", " ", x_str)

    # remove http links
    x_str = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x_str)

    # remove puncuation
    x_str = "".join([ci for ci in x_str if ci not in string.punctuation])

    # tokenise
    tokens = nltk.word_tokenize(x_str)

    # stem
    porter = nltk.PorterStemmer()
    stemmed_tokens = [porter.stem(token) for token in tokens]

    # remove stop words
    stopped_tokens = [ti for ti in stemmed_tokens if ti not in stopwords.words("english")]

    # remove MBTI types
    MBTI_types = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
                  'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
    MBTI_types = [ti.lower() for ti in MBTI_types]

    final_tokens = [wi for wi in stopped_tokens if wi not in MBTI_types]

    if return_joined:
        return " ".join([ci for ci in final_tokens])

    return final_tokens

In [11]:
df['processed_post'] = df['posts'].apply(lambda x:preprocess_string(x,True))

In [12]:
df.head()

,type,posts,IE,NS,TF,JP,processed_post
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,1,1,0,1,moment sportscent top ten play prank ha lifech...
1,ENTP,'I'm finding the lack of me in these posts ver...,0,1,1,0,im find lack post veri alarm sex bore posit of...
2,INTP,'Good one _____ https://www.youtube.com/wat...,1,1,1,0,good one cours say know bless curs doe absolut...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",1,1,1,1,dear enjoy convers day esoter gab natur univer...
4,ENTJ,'You're fired.|||That's another silly misconce...,0,1,1,1,fire anoth silli misconcept approach logic go ...


In [15]:
#df.to_csv (r'export_dataframe.csv', index = None, header=True)

## Train Test Split

In [16]:
from sklearn.model_selection import train_test_split

In [22]:
X = df['processed_post']
y = df['type']

y_IE = df['IE']
y_NS = df['NS']
y_TF = df['TF']
y_JP = df['JP']

In [18]:
# splitting the training and test set, 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [23]:
X_train, X_test, y_ie_train, y_ie_test = train_test_split(X, y_IE, test_size=0.3, random_state = 101)

In [41]:
X_train, X_test, y_ns_train, y_ns_test = train_test_split(X, y_NS, test_size=0.3, random_state = 101)

In [25]:
X_train, X_test, y_tf_train, y_tf_test = train_test_split(X, y_TF, test_size=0.3, random_state = 101)

In [44]:
X_train, X_test, y_jp_train, y_jp_test = train_test_split(X, y_JP, test_size=0.3, random_state = 101)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [21]:
from sklearn.pipeline import Pipeline

In [32]:
from sklearn.metrics import classification_report, confusion_matrix

## Naive Bayes Classifier

In [20]:
from sklearn.naive_bayes import MultinomialNB

In [30]:
nb_pipeline = Pipeline([
    ('vector',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('Classifier',MultinomialNB())
])

In [38]:
nb_pipeline.fit(X_train, y_jp_train)

Pipeline(memory=None,
         steps=[('vector',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('Classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [39]:
predictions = nb_pipeline.predict(X_test)

In [40]:
print(classification_report(y_jp_test, predictions))

              precision    recall  f1-score   support

           0       0.61      1.00      0.76      1581
           1       0.00      0.00      0.00      1022

    accuracy                           0.61      2603
   macro avg       0.30      0.50      0.38      2603
weighted avg       0.37      0.61      0.46      2603



/Users/mehulnagpurkar/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mehulnagpurkar/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mehulnagpurkar/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
rf_pipeline_1 = Pipeline([
    ('vector',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('Classifier',RandomForestClassifier(n_estimators=10))
])

In [59]:
rf_pipeline_1.fit(X_train, y_ie_train)

Pipeline(memory=None,
         steps=[('vector',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=N...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0

In [60]:
predictions_1 = rf_pipeline_1.predict(X_test)

In [61]:
print(classification_report(y_ie_test, predictions_1))

              precision    recall  f1-score   support

           0       0.29      0.08      0.13       604
           1       0.77      0.94      0.85      1999

    accuracy                           0.74      2603
   macro avg       0.53      0.51      0.49      2603
weighted avg       0.66      0.74      0.68      2603



In [62]:
rf_pipeline_2 = Pipeline([
    ('vector',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('Classifier',RandomForestClassifier(n_estimators=10))
])

In [63]:
rf_pipeline_2.fit(X_train, y_ns_train)

Pipeline(memory=None,
         steps=[('vector',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=N...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0

In [64]:
predictions_2 = rf_pipeline_2.predict(X_test)

In [65]:
print(classification_report(y_ns_test, predictions_2))

              precision    recall  f1-score   support

           0       0.18      0.01      0.03       343
           1       0.87      0.99      0.93      2260

    accuracy                           0.86      2603
   macro avg       0.52      0.50      0.48      2603
weighted avg       0.78      0.86      0.81      2603



In [66]:
rf_pipeline_3 = Pipeline([
    ('vector',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('Classifier',RandomForestClassifier(n_estimators=10))
])

In [67]:
rf_pipeline_3.fit(X_train, y_tf_train)

Pipeline(memory=None,
         steps=[('vector',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=N...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0

In [68]:
predictions_3 = rf_pipeline_3.predict(X_test)

In [69]:
print(classification_report(y_tf_test, predictions_3))

              precision    recall  f1-score   support

           0       0.60      0.80      0.69      1390
           1       0.63      0.39      0.49      1213

    accuracy                           0.61      2603
   macro avg       0.62      0.60      0.59      2603
weighted avg       0.62      0.61      0.59      2603



In [70]:
rf_pipeline_4 = Pipeline([
    ('vector',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('Classifier',RandomForestClassifier(n_estimators=10))
])

In [71]:
rf_pipeline_4.fit(X_train, y_jp_train)

Pipeline(memory=None,
         steps=[('vector',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=N...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0

In [72]:
predictions_4 = rf_pipeline_4.predict(X_test)

In [73]:
print(classification_report(y_jp_test, predictions_4))

              precision    recall  f1-score   support

           0       0.62      0.87      0.72      1581
           1       0.47      0.18      0.26      1022

    accuracy                           0.60      2603
   macro avg       0.55      0.52      0.49      2603
weighted avg       0.56      0.60      0.54      2603



## Checking against personality

In [91]:
pred_array = np.array([predictions_1, predictions_2, predictions_3, predictions_4])

In [92]:
pred_df = pd.DataFrame(pred_array.T)

In [86]:
pred_df['y_test'] = pd.DataFrame(y_test.T)

In [93]:
pred_df.head()

,0,1,2,3
0,1,1,1,0
1,1,1,0,0
2,1,1,0,0
3,0,1,0,0
4,1,1,0,0


In [106]:
pred_list = []
for tup in zip(predictions_1, predictions_2, predictions_3, predictions_4):
    mystring = []
    if tup[0] == 1:
        mystring += 'I'
    else:
        mystring += 'E'
        
    if tup[1] == 1:
        mystring += 'N'
    else:
        mystring += 'S'
        
    if tup[2] == 1:
        mystring += 'T'
    else:
        mystring += 'F'
        
    if tup[3] == 1:
        mystring += 'J'
    else:
        mystring += 'P'
        
    pred_list.append(''.join(mystring))

In [109]:
pred_array = np.array(pred_list)

In [110]:
print(classification_report(y_test, pred_array))

              precision    recall  f1-score   support

        ENFJ       0.04      0.02      0.03        52
        ENFP       0.06      0.03      0.04       214
        ENTJ       0.00      0.00      0.00        65
        ENTP       0.09      0.02      0.03       206
        ESFJ       0.00      0.00      0.00        10
        ESFP       0.00      0.00      0.00        13
        ESTJ       0.00      0.00      0.00         7
        ESTP       0.00      0.00      0.00        27
        INFJ       0.15      0.08      0.11       437
        INFP       0.21      0.54      0.31       577
        INTJ       0.09      0.04      0.05       314
        INTP       0.14      0.22      0.17       375
        ISFJ       0.00      0.00      0.00        52
        ISFP       0.00      0.00      0.00        88
        ISTJ       0.00      0.00      0.00        64
        ISTP       0.12      0.01      0.02       102

    accuracy                           0.17      2603
   macro avg       0.06   

/Users/mehulnagpurkar/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mehulnagpurkar/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mehulnagpurkar/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [111]:
y_test[:10]

893     INTP
752     INFJ
7296    INTJ
5647    INTP
8509    INTJ
7938    ISTJ
3320    ISFP
748     INTP
5956    ISTP
469     INFP
Name: type, dtype: object

In [112]:
pred_list[:10]

['INTP',
 'INFP',
 'INFP',
 'ENFP',
 'INFP',
 'INFP',
 'ENTP',
 'INFP',
 'INFP',
 'INFJ']

In [113]:
pred_df = pd.DataFrame(pred_array.T)

In [114]:
pred_df['y_test'] = pd.DataFrame(y_test.T)

In [117]:
#pred_df.columns = ['predictions', 'y_test']

In [118]:
pred_df.head()

,predictions,y_test
0,INTP,INFJ
1,INFP,NaN
2,INFP,NaN
3,ENFP,INTJ
4,INFP,NaN


In [122]:
len(y_test)

2603

In [123]:
from collections import defaultdict

correct_predictions = defaultdict(int)

for tup in zip(pred_array, y_test):
    if tup[0] == tup[1]:
        correct_predictions[tup[0]] += 1

In [124]:
correct_predictions

defaultdict(int,
            {'INTP': 81,
             'INFJ': 37,
             'INFP': 312,
             'INTJ': 11,
             'ENTP': 4,
             'ENFP': 6,
             'ENFJ': 1,
             'ISTP': 1})

In [126]:
sum(correct_predictions.values())

453

In [127]:
sum(correct_predictions.values())/len(y_test)

0.17402996542451019

In [128]:
# try to represent the probability of personality on some n space
# give a list of potential personality types, and commonalities in that